In [77]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import urllib.request
from bs4 import BeautifulSoup
import bs4
import lxml
url = "https://en.wikipedia.org/wiki/List_of_Metroid_media"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'lxml')

In [78]:
print(soup.title)
soup.title.string
i = soup.findAll('i')
gameSeires = []
gameSeires.append(i[0].text + i[0].nextSibling.text)
gameSeires.append(i[1].text + i[1].nextSibling.text)
print(gameSeires)

<title>List of Metroid media - Wikipedia</title>
['Metroid series', 'Metroid Prime series']


In [79]:
for junk in soup.findAll('sup'):
    junk.extract()

In [80]:
#I REALLY wanna merge them and make my code smaller, more compact, and cooler but this is more readable :( 
counter = 0
i = soup.findAll('i')
for table in soup.find_all("table", class_='wikitable plainrowheaders video-game-titles'):
    gameName = []
    for row in table.findAll('tr',class_="hproduct"):
        for name in row.findAll('th'):
            gameName.append(name.find(string=True))

    releaseDatesregions = []
    for row in table.findAll('tr',class_="hproduct"):#this is reduculus, 4 nested loops. There has to be a better way.
        for name in row.findAll(class_='plainlist'):
            dates = {}
            for li in name.findAll('ul'):
                for item in li.findAll('li'):
                    for junk in item.findAll('a'):
                        region = junk.text
                        junk.extract()
                    for junk in item.findAll('abbr'):
                        region = junk.text
                        junk.extract()
                    for junk in item.findAll('span'):
                        junk.extract()
                    dates[region] = item.text.strip()
                releaseDatesregions.append(dates)

    notes = []
    for row in table.findAll('tr'):
        if 'class' not in row.attrs:
            for b in row.findAll('b'):
                b = b.text #using b as dictionary key cuz it funny
            noteList = []
            for li in row.findAll('li'):
                noteList.append(li.text)
            if noteList != [] or len(notes) == 8:#I gave up
                notes.append(noteList)

    releaseDatesSystem = []
    for row in table.findAll('tr',class_='hproduct'):
        gameSystemRelease = {}
        for td in row.findAll('td'):
            for b in td.findAll('b'):
                B = b
                b.extract()
            tempSoup = BeautifulSoup(str(td).replace('<br/>','\n'), 'lxml')#why did I pick this website
            systems = tempSoup.text.strip().split('\n')
            for i in systems:
                if i != ':' and i != '':        
                    temp = i.split(' – ')#why does the html use a different unicode character then (-). uggggggggghh
                    if len(temp) == 1:#this took me 2 hours to figure out just because a single var is just left BLANK WHY?!?!?!?!?!??! Im about to commit a war crime
                        temp = i.split('–')#why did the person who made this html use a different unicode character then (-). uggggggggghh
                        gameSystemRelease['TBD'] = temp[1].strip()
                    else:
                        gameSystemRelease[temp[0].strip()] = temp[1].strip()
            releaseDatesSystem.append(gameSystemRelease)
    fileName = f'{gameSeires[counter]}.cvs'
    games = pd.DataFrame(data={'game title' : gameName, 'region release dates': releaseDatesregions, str(B.text): releaseDatesSystem, 'notes': notes})
    games.to_csv(fileName, sep='|')
    counter += 1

Add code below for graphs